In [ ]:
'''
Exploratory Data Analysis and Model Training for Taxi Fare Prediction

This Jupyter Notebook supports initial data exploration and trains a Random Forest Regression 
model to predict taxi fares based on various input features. It includes steps for data extraction, 
preprocessing, feature engineering, model training, and evaluation.
'''
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Load and Filter Data
# We're only interested in trips to LaGuardia Airport,
# so we'll create a new dataset for just those rows.
# We calculate total cost, and discard outliers with negative costs.
# We discard irrelevant columns to reduce dataset to minimal size.
# This reduces the dataset from ~1GB combined to only ~9MB for 900k rows.
# ---
# Trip data source: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
# Dataset titles: "High Volume For-Hire Vehicle Trip Records"
# Dataset URLs:
# April 2024, https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-04.parquet
# May 2024, https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2024-05.parquet
# ---

# File Paths
input_files = [
    'fhvhv_tripdata_2024-04.parquet',
    'fhvhv_tripdata_2024-05.parquet'
]
output_file = 'filtered.parquet'
borough_file = 'taxi_boroughs.csv' # output of parse_taxi_zones.py

# Process Input Data
# Read and process each input Parquet file
dataframes = []
for input_file in input_files:
    raw = pd.read_parquet(input_file)

    # Select features relevant for trip cost
    columns_to_keep = ['pickup_datetime', 'PULocationID', 'DOLocationID']

    # Select only trips to LGA – Taxi Zone 138, then drop the column
    filtered_data = raw.loc[raw['DOLocationID'] == 138, columns_to_keep]
    filtered_data = filtered_data.drop(columns=['DOLocationID'])

    # Add column for sum of variable costs for each ride
    filtered_data['total_cost'] = (
        raw['base_passenger_fare'] +
        raw['tolls'] +
        raw['congestion_surcharge']
    )

    # Remove negative outliers
    filtered_data = filtered_data[filtered_data['total_cost'] >= 0]

    # Append processed dataframe to list
    dataframes.append(filtered_data)

# Combine input dataframes into one
combined_data = pd.concat(dataframes, ignore_index=True)

# Merge Borough Data
# Read borough data
boroughs = pd.read_csv(borough_file)
# Merge borough names into dataset on matching Taxi Zone number
combined_data = combined_data.merge(boroughs[['OBJECTID', 'borough']], left_on='PULocationID', right_on='OBJECTID').drop(columns=['OBJECTID'])

# Save Working Dataset
# Save to new working dataset for graphs and models to follow
combined_data.to_parquet(output_file, index=False)

In [ ]:
# Summarize Dataset
data = pd.read_parquet(output_file)
print(data.head())
print(data.info())
print(data.describe())

In [ ]:
# Handling Outliers

# Calculate 25th and 75th percentile boundaries
Q1 = data['total_cost'].quantile(0.25)
Q3 = data['total_cost'].quantile(0.75)
IQR = Q3 - Q1

# Define cutoff point for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
data = data[(data['total_cost'] >= lower_bound) & (data['total_cost'] <= upper_bound)]

# Summarize again to check the results
print(lower_bound)
print(upper_bound)
print(data.head())
print(data.info())
print(data.describe())


In [ ]:
# Transform Data for Visualization and Modeling

# Bin datetimes by hour of day
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime']).dt.hour
data.rename(columns={'pickup_datetime': 'hour_of_day'}, inplace=True)

# Replace categorical Taxi Zone numbers and borough names with T/F dummy variables
data = pd.get_dummies(data, columns=['PULocationID', 'borough'])

# Save the list of feature_names for use in app.py
feature_names = data.drop(
    columns=['total_cost',
             'borough_Bronx',
             'borough_Brooklyn',
             'borough_Manhattan',
             'borough_Queens',
             'borough_Staten Island']).columns.tolist()
joblib.dump(feature_names, 'feature_names.pkl')

# Sanity check
print(data.head())
print(data.info())

In [ ]:
# Visualize Data

# Average cost by borough
# Notable: Staten Island has highest base cost due to greatest distance
# Notable: Queens has lowest base cost due to shortest distance
plt.figure(figsize=(12, 6))
graph = sns.catplot(x="total_cost", y="borough", order=['Bronx','Brooklyn','Manhattan','Queens','Staten Island'], kind="boxen", height=6, aspect=2, data=combined_data)
graph.fig.suptitle('Average Cost by Borough', y=1.03) # adjusting to avoid cutting off title
graph.set_axis_labels("Average Fare", "Borough")  # Set axis labels
graph.savefig('static/catplot_avg_cost_by_borough.png')
plt.show()
plt.close()

In [ ]:
# Top pickup locations
# Notable: had to use a log scale to see Staten Island
# Notable: Staten Island count reduced due to exclusion of pricy outliers
borough_dummies = ['borough_Bronx', 'borough_Brooklyn', 'borough_Manhattan', 'borough_Queens', 'borough_Staten Island']

pickup_counts = data[borough_dummies].sum().sort_values(ascending=False).head(20)
pickup_counts.index = pickup_counts.index.str.replace('borough_', '') # rename the columns

plt.figure(figsize=(12, 6))
ax = sns.barplot(x=pickup_counts.values, y=pickup_counts.index)
plt.xscale('log')
plt.title('Top Pickup Locations')
plt.xlabel('Count')
plt.ylabel('Pickup Location')
ax.bar_label(ax.containers[0], fontsize=8)
plt.savefig('static/barplot_top_pickup_locations.png')
plt.show()
plt.close()

In [ ]:
# Distribution of total cost
# Notable: dip at ~$15-25 range, possible impact of tolls on similar-distance trips
# Notable: very, very long tail before removing outliers
plt.figure(figsize=(10,6))
sns.histplot(data['total_cost'], bins=50, kde=True)
plt.title('Distribution of Fares')
plt.xlabel('Total Cost')
plt.ylabel('Count')
plt.savefig('static/histplot_distribution_of_fares.png')
plt.show()
plt.close()

In [ ]:
# Total cost by hour of day
plt.figure(figsize=(10, 6))
sns.boxplot(x='hour_of_day', y='total_cost', data=data)
plt.title('Cost by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Cost')
plt.savefig('static/boxplot_cost_by_hour_of_day.png')
plt.show()
plt.close()

In [ ]:
# Average cost by pickup hour
# Notable: greater variance at night
plt.figure(figsize=(10,6))
sns.barplot(x='hour_of_day', y='total_cost', data=data)
plt.title('Average Cost by Pickup Hour')
plt.xlabel('Pickup Hour')
plt.ylabel('Cost')
plt.xticks(rotation=0)
plt.savefig('static/barplot_avg_cost_by_pickup_hour.png')
plt.show()
plt.close()

In [ ]:
# Train Two Models

# We're predicting fares at two levels of granularity: borough and neighborhood (Taxi Zone).
# We'll use boroughs for our simplest baseline model.
# Then compare with the neighborhood model to see if more features improve predictive power.

# Before we create models, we need to make sure each is using only the relevant features.
# We'll identify the columns that need to be dropped.
borough_columns = [col for col in data.columns if col.startswith('borough')]
neighborhood_columns = [col for col in data.columns if col.startswith('PULocationID')]

# Borough data should exclude the neighborhoods
# Neighborhood data should exclude the boroughs
borough_data = data.drop(columns=neighborhood_columns)
neighborhood_data = data.drop(columns=borough_columns)

In [ ]:
# Train and Evaluate Borough Model

# Prepare data
X_borough = borough_data.drop(columns=['total_cost'])
y_borough = borough_data['total_cost']

# Split data
X_train_borough, X_test_borough, y_train_borough, y_test_borough = train_test_split(X_borough, y_borough, test_size=0.2, random_state=38)

# Display training set to verify splits
print(X_train_borough.head())

# Train the model
rf_borough = RandomForestRegressor(random_state=38)
rf_borough.fit(X_train_borough, y_train_borough)

# Predict and evaluate
y_pred_borough = rf_borough.predict(X_test_borough)
mae_borough = mean_absolute_error(y_test_borough, y_pred_borough)
mse_borough = mean_squared_error(y_test_borough, y_pred_borough)
r2_borough = r2_score(y_test_borough, y_pred_borough)

# Print performance summary
print(f"Borough Model - MAE: {mae_borough}, MSE: {mse_borough}, R²: {r2_borough}")

# Performance Results for Borough Model
# Model 1, Initial Run
# MAE: 11.858519352878245, MSE: 268.929452004129, R²: 0.442588901962921
# Model 2, After Removing Outliers
# MAE: 10.077229327066409, MSE: 167.0615420693796, R²: 0.5103575632105499
# Model 3, After Combining Two Months' Parquet Files
# MAE: 10.49845986493553, MSE: 183.0367370351296, R²: 0.510364551383309

In [ ]:
# Train and Evaluate Neighborhood Model

# Prepare data
X_neighborhood = neighborhood_data.drop(columns=['total_cost'])
y_neighborhood = neighborhood_data['total_cost']

# Split data
X_train_neighborhood, X_test_neighborhood, y_train_neighborhood, y_test_neighborhood = train_test_split(X_neighborhood, y_neighborhood, test_size=0.2, random_state=38)

# Display training set to verify splits
print(X_train_neighborhood.head())

# Train the model
rf_neighborhood = RandomForestRegressor(random_state=38)
rf_neighborhood.fit(X_train_neighborhood, y_train_neighborhood)

# Predict and evaluate
y_pred_neighborhood = rf_neighborhood.predict(X_test_neighborhood)
mae_neighborhood = mean_absolute_error(y_test_neighborhood, y_pred_neighborhood)
mse_neighborhood = mean_squared_error(y_test_neighborhood, y_pred_neighborhood)
r2_neighborhood = r2_score(y_test_neighborhood, y_pred_neighborhood)

# Print performance summary
print(f"Neighborhood Model - MAE: {mae_neighborhood}, MSE: {mse_neighborhood}, R²: {r2_neighborhood}")

# Performance Results for Neighborhood Model
# Model 1, Initial Run
# MAE: 9.033939139834594, MSE: 186.04478396046923, R²: 0.6143842686672932
# Model 2, After Removing Outliers
# MAE: 7.367526755657218, MSE: 102.62094150209452, R²: 0.6992271995080372
# Model 3, After Combining Two Months' Parquet Files
# MAE: 7.700498524423761, MSE: 113.15572572538952, R²: 0.6973009056730255

In [ ]:
# Save Trained Model
model_filename = 'modelv3.pkl'
joblib.dump(rf_neighborhood, model_filename)